In [46]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using device: {device}")

Using device: mps


In [47]:
block_size = 8
batch_size = 4
max_iters = 1000
learning_rate = 3e-4
eval_iters = 250

In [48]:
with open ('wizard_of_oz.txt', 'r', encoding='utf-8') as f: 
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '(', ')', ',', '-', '.', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '—', '‘', '’', '“', '”', '\ufeff']


In [49]:
string_to_int = { c: i for i, c in enumerate(chars) }
int_to_string = { i: c for i, c in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([68, 13, 44, 37, 52, 56, 41, 54,  1, 19,  0, 30, 44, 41,  1, 13, 61, 39,
        48, 51, 50, 41,  0,  0,  0, 14, 51, 54, 51, 56, 44, 61,  1, 48, 45, 58,
        41, 40,  1, 45, 50,  1, 56, 44, 41,  1, 49, 45, 40, 55, 56,  1, 51, 42,
         1, 56, 44, 41,  1, 43, 54, 41, 37, 56,  1, 21, 37, 50, 55, 37, 55,  1,
        52, 54, 37, 45, 54, 45, 41, 55,  5,  1, 59, 45, 56, 44,  1, 31, 50, 39,
        48, 41,  0, 18, 41, 50, 54, 61,  5,  1])


In [50]:
n = int(0.8 * len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i + block_size] for i in ix])
    y = torch.stack([data[i + 1:i + block_size + 1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print("Inputs: ", x)
print("Targets: ", y)

Inputs:  tensor([[55, 37, 56,  1, 38, 41, 44, 45],
        [56, 44,  1, 52, 51, 45, 50, 56],
        [41,  1, 51, 42,  1, 51, 50, 41],
        [56, 51,  1, 56, 44, 41,  1, 55]], device='mps:0')
Targets:  tensor([[37, 56,  1, 38, 41, 44, 45, 50],
        [44,  1, 52, 51, 45, 50, 56, 41],
        [ 1, 51, 42,  1, 51, 50, 41,  1],
        [51,  1, 56, 44, 41,  1, 55, 56]], device='mps:0')


In [51]:
block_size = 8

x = train_data[:block_size]
y = train_data[1:block_size + 1]

for t in range(block_size):
    context = x[:t + 1]
    target = y[t]
    print(f"when input is {context}, the target: {target}")

when input is tensor([68]), the target: 13
when input is tensor([68, 13]), the target: 44
when input is tensor([68, 13, 44]), the target: 37
when input is tensor([68, 13, 44, 37]), the target: 52
when input is tensor([68, 13, 44, 37, 52]), the target: 56
when input is tensor([68, 13, 44, 37, 52, 56]), the target: 41
when input is tensor([68, 13, 44, 37, 52, 56, 41]), the target: 54
when input is tensor([68, 13, 44, 37, 52, 56, 41, 54]), the target: 1


In [52]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets= None):
        logits = self.token_embedding_table(idx)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self.forward(idx)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx
    
model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


FLAzkbyHWg﻿zsVkX!J?wHkjoOAy
﻿IE?wGdCvsmm!i(“dSGhdV
“.X?K﻿AZn:.e,P﻿M:F)Ltk﻿;Vq.
gUcKmz?v“FbQdt‘UwshCKm”uAh
g;-atPT,dtX“?Ck”’—xwyZvMeVeeLL”xvPuKijJxmkJBX”FGq“XIkPXl—wxa)r?“m)Y,-pA’R‘n ujcYIdVJ‘SoGNBfCb,wZtq..
lm-drY“X-bltB!;blSsXkkMx!Hq!X’iac(F—SjoQAosv!‘n)L”khWfshAB’zOjoR(HCT(cb:n—Vk?vMB’m!“gGCEWz,“y
QwN?vshMPSGrX”XlZc(Bzsu(g﻿dp’yhxQQG) —nwQdorZxbQFE,DDy)rZBz
gi’PgmflBCpXs’RVJ’(,—sG)TlE﻿Gor“dKGCs.dCZZCv“Y‘?“rUS‘AixaMt
“—gB’RyNNV-;PrOViSEn,’MgtNkFi.UKmg).wNT!UQwNk;UKMBzcFVTa,P-RyCU)IN“HMeCAz?wZDjo


In [53]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            x, y = get_batch(split)
            logits, loss = model(x, y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [58]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f'step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}')
    xb, yb = get_batch('train')
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(f"Iteration {iter}, Loss: {loss.item()}")

step: 0, train loss: 3.940, val loss: 3.986
step: 250, train loss: 3.909, val loss: 3.907
step: 500, train loss: 3.871, val loss: 3.879
step: 750, train loss: 3.816, val loss: 3.809
Iteration 999, Loss: 3.415341377258301


In [55]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


dn:CmZhEqTBFqc’Xlu-xwZMTY.v.
SGJ jyCWqrZnraULzIdqnL‘EXoMWvAOGjLUwVsmcK??ff)WFC﻿.LpzzjXvM“R’-SDXee HmrZdXBCfIicSaaDJEkwW-BJXYX
tx!fdC﻿sclFYi,p-qdKT—‘nA”r?HFse.fIb!Zn

gTVkbe?JFrYbe‘DBz?-n:Lt“hfsEUQgQruzMBzsmlL fuce
nm
q(?Eie?l(LySL ‘)r?(BCP;’QG.wzslAixzsnJZHf;K
dB“eeR(p’iezctBzX?“Ltn:DHfEdCY—raio—rZnQgA-HkkjOdKTNn:LMF(QYSd?a wO,l;TSL”xdhwky(L‘﻿VI.”-HNunqDTVIHVf﻿G.gGYpKA”;.dqJ:FpZn—KSxG;M(B“cmxupeMgTy
qnUQZ;BSxBz-kJZX!gK‘﻿﻿XSg
s
hT.gp’QA?v?HEeFwN-R(oQQ.G—;﻿Kp(pacosmm:-n!gYY)IP;ZN?f?)U”B)Om-nCfjgip
